In [ ]:
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import numpy as np
# 假设这是你之前的 image_loader 模块
# import image_loader 

class RatioApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Ratio Imaging Analyzer - Import Logic")
        self.root.geometry("500x400")

        # --- 数据模型 (Model) ---
        # 无论从哪里导入，最终数据都存这里
        self.img_ch1 = None  # 分子 (e.g. 340nm)
        self.img_ch2 = None  # 分母 (e.g. 380nm)

        # --- 界面布局 (View) ---
        self.setup_ui()

    def setup_ui(self):
        # 1. 创建导入区域容器
        # 改为 ttk.LabelFrame
        import_frame = ttk.LabelFrame(self.root, text="数据导入 (Data Import)", padding=10)
        import_frame.pack(fill="x", padx=10, pady=10)

        # 2. 使用选项卡 (Notebook) 区分两种模式
        self.notebook = ttk.Notebook(import_frame)
        self.notebook.pack(fill="both", expand=True)

        # --- Tab 1: 分别导入 ---
        tab_separate = ttk.Frame(self.notebook)
        self.notebook.add(tab_separate, text=' 分别导入 (两文件) ')
        
        btn_ch1 = ttk.Button(tab_separate, text="导入通道 1 (340nm)...", command=self.load_separate_ch1)
        btn_ch1.pack(pady=5, fill='x')
        
        btn_ch2 = ttk.Button(tab_separate, text="导入通道 2 (380nm)...", command=self.load_separate_ch2)
        btn_ch2.pack(pady=5, fill='x')

        # --- Tab 2: 合并导入 ---
        tab_combined = ttk.Frame(self.notebook)
        self.notebook.add(tab_combined, text=' 双通道文件导入 (单文件) ')

        # 针对合并文件的额外选项
        self.is_interleaved = tk.BooleanVar()
        chk_inter = ttk.Checkbutton(tab_combined, text="使用交错拆分 (Interleaved)", variable=self.is_interleaved)
        chk_inter.pack(pady=5)

        btn_dual = ttk.Button(tab_combined, text="导入双通道 Stack...", command=self.load_combined_file)
        btn_dual.pack(pady=5, fill='x')
        
        btn_swap = ttk.Button(tab_combined, text="交换通道顺序 (Swap Ch1/Ch2)", command=self.swap_channels)
        btn_swap.pack(pady=5)

        # 3. 状态反馈区域 (显示是否准备就绪)
        status_frame = ttk.LabelFrame(self.root, text="当前状态", padding=10)
        status_frame.pack(fill="x", padx=10, pady=10)

        self.lbl_status_ch1 = tk.Label(status_frame, text="通道 1: 未加载", fg="red")
        self.lbl_status_ch1.pack(anchor="w")
        
        self.lbl_status_ch2 = tk.Label(status_frame, text="通道 2: 未加载", fg="red")
        self.lbl_status_ch2.pack(anchor="w")

        # 4. 确认/分析按钮
        self.btn_analyze = ttk.Button(self.root, text="开始分析 (Start Analysis)", state="disabled", command=self.start_analysis)
        self.btn_analyze.pack(pady=20)

    # --- 逻辑控制 (Controller) ---

    def update_status(self):
        """检查两个通道是否都齐了，更新界面"""
        # 检查通道1
        if self.img_ch1 is not None:
            self.lbl_status_ch1.config(text=f"通道 1: 已就绪 (Shape: {self.img_ch1.shape})", fg="green")
        else:
            self.lbl_status_ch1.config(text="通道 1: 未加载", fg="red")
            
        # 检查通道2
        if self.img_ch2 is not None:
            self.lbl_status_ch2.config(text=f"通道 2: 已就绪 (Shape: {self.img_ch2.shape})", fg="green")
        else:
            self.lbl_status_ch2.config(text="通道 2: 未加载", fg="red")

        # 只有两个都齐了，才让点“开始分析”
        if self.img_ch1 is not None and self.img_ch2 is not None:
            self.btn_analyze.config(state="normal")
        else:
            self.btn_analyze.config(state="disabled")

    def load_separate_ch1(self):
        path = filedialog.askopenfilename(title="选择通道1文件")
        if path:
            # 模拟读取
            # self.img_ch1 = image_loader.read_file(path) 
            self.img_ch1 = np.zeros((10, 512, 512)) # 假数据
            print(f"Loaded Ch1 from {path}")
            self.update_status()

    def load_separate_ch2(self):
        path = filedialog.askopenfilename(title="选择通道2文件")
        if path:
            # 模拟读取
            self.img_ch2 = np.zeros((10, 512, 512)) # 假数据
            print(f"Loaded Ch2 from {path}")
            self.update_status()

    def load_combined_file(self):
        path = filedialog.askopenfilename(title="选择双通道文件")
        if path:
            try:
                # 模拟调用 image_loader 的拆分功能
                # ch1, ch2 = image_loader.read_and_split_file(path, self.is_interleaved.get())
                
                # 这里用假数据模拟拆分成功
                print("Splitting file...")
                ch1 = np.zeros((10, 512, 512)) 
                ch2 = np.ones((10, 512, 512)) 
                
                # 一次性填充两个槽位
                self.img_ch1 = ch1
                self.img_ch2 = ch2
                
                messagebox.showinfo("成功", "双通道文件已自动拆分！")
                self.update_status()
                
            except Exception as e:
                messagebox.showerror("错误", str(e))

    def swap_channels(self):
        """仅在合并导入模式下有用：交换 Ch1 和 Ch2"""
        if self.img_ch1 is None or self.img_ch2 is None:
            return
        
        # 交换数据
        self.img_ch1, self.img_ch2 = self.img_ch2, self.img_ch1
        messagebox.showinfo("交换", "通道顺序已交换。\n现在 Ch1 是原来的 Ch2。")
        self.update_status()

    def start_analysis(self):
        print("进入分析流程...")

if __name__ == "__main__":
    root = tk.Tk()
    app = RatioApp(root)
    root.mainloop()

TclError: unknown option "-padding"